In [ ]:
#import pandas as pd
import numpy as np
import plotly.express as px
import experiment_helpers

from importlib import reload

from experiment_helpers import *
reload(experiment_helpers)

In [ ]:
EXPERIMENT_TYPES = [
    "no_prefetching",
    "linux_prefetching",
    #"linux_prefetching_ssdopt",
    #"linux_prefetching_ssdopt_asyncwrites","linux_prefetching_ssdopt_asyncwrites",
    #"tape_prefetching_asyncwrites","tape_prefetching_syncwrites"
]
annotations=[]

In [ ]:
def add_annotation(text):
    anno = "<h3><pre>%s</pre></h3>" % text
    annotations.append(anno)

# Experiment plots

In [ ]:
WORKLOADS = ["mmult_eigen","vec_eigen", "dot_eigen", "bitonic_merge", "bitonic_sort", "mmap_random_rw", "kissfft", "kmeans"]
#"alexnet", "vgg16", "batcher_sort"]:
EXPERIMENT_TYPES = ["no_prefetching", "linux_prefetching"]


In [ ]:
ls ../experiment_results

In [ ]:
aux = lambda w, TYPE: (w,augment_tables(get_experiment_data(EXPERIMENT_TYPES, w,"../experiment_results"), True)[EXPERIMENT_TYPES.index(TYPE)])
                          
named_dfs_no_prefetching = [aux(w,"no_prefetching") for w in WORKLOADS]                                                            
named_dfs_linux_prefetching = [aux(w,"no_prefetching") for w in WORKLOADS]                                                            

In [ ]:
plots = []
for workload, named_dfs in [("no_prefetching",named_dfs_no_prefetching), 
                            ("linux_prefetching",named_dfs_linux_prefetching)]: 
    


    degradations = take_column_from_dfs("Degradation(wallclock)",named_dfs).dropna() * 100
    degradations_no_ev = take_column_from_dfs("Degradation w/o Evictions",named_dfs).dropna() * 100
    runtime = take_column_from_dfs("Measured(wallclock) runtime",named_dfs).dropna()


    fig = px.line(runtime, title='(%s)Absolute(Wallclock) Runtime vs. Local Memory'% workload)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Measured(wallclock) runtime(s)",
    )

    anno = """
    Things to note above: 
        1) simply skipping an ssd-specific code path in the kernel when allocating
            swap entries significantly improves eviction performance (top vs middle 2 lines)
        2)on top of that, adding tape prefetching (lowest 2 lines) further improves end to end performance
    """
    add_annotation(text=anno)
    fig.show()
    plots.append(fig)

    fig = px.line(degradations, title='(%s)Normalized Runtime vs. Local Memory' % workload)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Degradation(%)",
    )

    anno = """
    Things to note above: 
        1) performance differences among different fastswap configurations are not visible here since we are comparing
           performance of each system under limited memory to the performance of THE SAME system under 100%
        2) Even in case of good tape prefetching performance degrades (due to evictions!) see next plot

    """
    add_annotation(text=anno)
    fig.show()
    plots.append(fig)

    fig = px.line(degradations_no_ev, title='(%s)Normalized Runtime vs. Local Memory (SUBTRACT time spent in evictions)' % workload)
    fig.update_layout(
        xaxis_title="Ratio(%)",
        yaxis_title="Degradation(%)",
    )
    anno = """
    Things to note above: 
        1) If we subtract the time spent on handling eviction on application CPU from previous set of grapshs, we see
           that tape prefetching is a lot less dependent on local memory ratio, even when it is extremely small
        2) the remaining overhead of tape prefetching (about x2 degradation) is a result of inefficent code, can be optimized
           but as the contrast shows, optimizing evictions would give us a lot more

    """
    add_annotation(text=anno)
    fig.show()
    plots.append(fig)

In [ ]:
runtime_components = []
anno = """
Things to note above: 
    
    1) When looking at different components of what comprises end to end application runtime, we see again that
      evictions is where most time is spent
    2) (DISABLE evictions plot to see the result) when not considering evictions, we see that in the plot most time is
       spent on major page faults
       
"""
add_annotation(text=anno)

anno = """
Things to note above: 
    
    1) Unlike the previous case of linux prefetching, when we use tape to prefetch, we essentially eliminate time spent
       on major page faults
    2) we do not do any mapping to page tables hence the increase minor faults but this can be fixed for a SMALL runtime
       gain (w.r.t. more efficient evictions)
       
"""
add_annotation(text=anno)

#"tape_prefetching_syncwrites", "tape_prefetching_asyncwrites"
for workload in WORKLOADS:
    EXPERIMENT_TYPES = ["no_prefetching", "linux_prefetching"]
    experiment_results = augment_tables(get_experiment_data(EXPERIMENT_TYPES, workload,"../experiment_results"), True)
    named_dfs = list(zip(EXPERIMENT_TYPES, experiment_results))
    for exp_name in ["no_prefetching", "linux_prefetching"]:
        fig = get_components_of_runtime(get_experiment_table(named_dfs,exp_name), "%s,%s" % (exp_name, workload))
        runtime_components.append(fig)
    #    fig.update_layout(yaxis_range=[0,15])

        fig.show()

In [ ]:
for exp_name in ["tape_prefetching_syncwrites", "tape_prefetching_asyncwrites"]:
    fig = get_components_of_runtime(get_experiment_table(named_dfs,exp_name))
    add_annotation(text="tape sync writes anno")
    add_annotation(text="tape async writes anno")

    runtime_components.append(fig)
    fig.update_layout(yaxis_range=[0,15])
    fig.show()


In [ ]:
def spill_html():
    with open('linux.html', 'w') as f:
        i = 0
        for figures in [plots,runtime_components]:
            for figure in figures:
                #anno = annotations[i]
                #i+=1
                f.write(figure.to_html(full_html=False, include_plotlyjs='cdn'))
                #f.write(anno)
                f.write("<hr/>")

In [ ]:
spill_html()